### Test de l'api_brussel

In [1]:
# api_brussels_correct.py
import requests
import json

class BrusselsAPICorrect:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        """Génère un nouveau token d'accès"""
        print("🔄 Génération d'un nouveau token...")
        
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        
        try:
            response = requests.post(token_url, data=payload)
            print(f"Token Response Status: {response.status_code}")
            
            if response.status_code == 200:
                token_data = response.json()
                new_token = token_data["access_token"]
                expires_in = token_data.get("expires_in", "Inconnu")
                print(f"✅ NOUVEAU TOKEN GÉNÉRÉ!")
                print(f"   Token: {new_token[:20]}...")
                print(f"   Expire dans: {expires_in} secondes")
                return new_token
            else:
                print(f"❌ Erreur génération token: {response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Exception token: {e}")
            return None
    
    def get_events(self, page=1, limit=10):
        """Récupère les événements avec la bonne URL"""
        if not self.access_token:
            print("❌ Aucun token disponible")
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        # URL CORRECTE comme dans la documentation
        url = f"{self.base_url}/events"
        params = {"page": page}
        
        print(f"🔍 Appel API: {url}")
        print(f"   Token: {self.access_token[:20]}...")
        
        try:
            response = requests.get(url, headers=headers, params=params)
            print(f"📡 Status Code: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                events = data.get("events", [])
                print(f"✅ SUCCÈS! {len(events)} événements récupérés")
                return events[:limit]  # Limite le nombre
            else:
                print(f"❌ ERREUR API: {response.text}")
                return []
                
        except Exception as e:
            print(f"❌ Exception: {e}")
            return []
    
    def search_events(self, keyword: str):
        """Recherche des événements par mot-clé"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events/search"
        params = {"keyword": keyword}
        
        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 200:
                return response.json().get("events", [])
            else:
                print(f"Erreur recherche: {response.status_code}")
                return []
        except Exception as e:
            print(f"Exception recherche: {e}")
            return []

# Test complet
if __name__ == "__main__":
    print("🧪 API BRUSSELS AGENDA - VERSION CORRIGÉE")
    print("=" * 60)
    
    api = BrusselsAPICorrect()
    
    if api.access_token:
        print("\n📅 RÉCUPÉRATION DES ÉVÉNEMENTS...")
        events = api.get_events(page=1, limit=5)
        
        if events:
            print(f"\n🎉 {len(events)} ÉVÉNEMENTS TROUVÉS:")
            print("-" * 50)
            
            for i, event in enumerate(events, 1):
                print(f"\n{i}. {event.get('title', 'Sans titre')}")
                print(f"   📍 {event.get('location', {}).get('name', 'Lieu non spécifié')}")
                print(f"   📅 {event.get('date', {}).get('start', 'Date non spécifiée')}")
                print(f"   🔗 {event.get('url', 'Pas de lien')}")
                
                # Affiche un extrait de la description
                description = event.get('description', '')
                if description:
                    print(f"   📝 {description[:100]}...")
        else:
            print("😞 Aucun événement trouvé")
            
        # Test de recherche
        print("\n🔍 TEST DE RECHERCHE 'concert'...")
        concerts = api.search_events("concert")
        print(f"🎵 {len(concerts)} concerts trouvés")
        
    else:
        print("❌ Impossible d'initialiser l'API")

🧪 API BRUSSELS AGENDA - VERSION CORRIGÉE
🔄 Génération d'un nouveau token...
Token Response Status: 200
✅ NOUVEAU TOKEN GÉNÉRÉ!
   Token: 097590bb-eca0-35c4-9...
   Expire dans: 9223372036854775 secondes

📅 RÉCUPÉRATION DES ÉVÉNEMENTS...
🔍 Appel API: https://api.brussels:443/api/agenda/0.0.1/events
   Token: 097590bb-eca0-35c4-9...
📡 Status Code: 200
✅ SUCCÈS! 0 événements récupérés
😞 Aucun événement trouvé

🔍 TEST DE RECHERCHE 'concert'...
Erreur recherche: 500
🎵 0 concerts trouvés


In [15]:
# api_brussels_fixed_v2.py
import requests
import json
from datetime import datetime

class BrusselsAPIWorking:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        # self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        """Génère un nouveau token d'accès"""
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, page=1, limit=10):
        """Récupère les événements avec la bonne structure"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        # url = f"{self.base_url}/events"
        url = f"{self.base_url}/events/category"
        # params = {"page": page}
        params = {"mainCategory": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            
            # EXTRACTION CORRECTE des événements
            events = data.get("response", {}).get("results", {}).get("event", [])
            
            print(f"📊 Statistiques:")
            print(f"   - Page: {data.get('response', {}).get('page', 1)}")
            print(f"   - Total pages: {data.get('response', {}).get('pageCount', 1)}")
            print(f"   - Total événements: {data.get('response', {}).get('resultCount', 0)}")
            print(f"   - Événements cette page: {len(events)}")
            
            return events[:limit]
        else:
            print(f"❌ Erreur API: {response.status_code}")
            return []
    
    def debug_event_structure(self, event):
        """Debug la structure d'un événement"""
        print("\n🔍 STRUCTURE DE L'ÉVÉNEMENT:")
        print(json.dumps(event, indent=2, ensure_ascii=False)[:1000] + "...")
    
    def format_event(self, event):
        """Formate un événement pour l'affichage"""
        # Debug si nécessaire
        # self.debug_event_structure(event)
        
        # Gère les traductions
        translations = event.get("translations", {})
        fr_translation = translations.get("fr", {})
        nl_translation = translations.get("nl", {})
        
        title = fr_translation.get("title") or nl_translation.get("title") or "Sans titre"
        description = fr_translation.get("shortdescr") or nl_translation.get("shortdescr") or ""
        
        # DATE - La structure est différente
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        end_date = ""
        
        if dates and isinstance(dates, list) and len(dates) > 0:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", "Date non spécifiée")
                end_date = first_date.get("end", "")
        
        # Lieu
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        # Prix
        price_info = event.get("price", {})
        if isinstance(price_info, dict):
            price = price_info.get("base", "Gratuit" if event.get("is_free", False) else "Payant")
        else:
            price = "Gratuit" if event.get("is_free", False) else "Payant"
        
        return {
            "title": title,
            "description": description[:200] + "..." if len(description) > 200 else description,
            "start_date": start_date,
            "end_date": end_date,
            "location": place_name,
            "url": fr_translation.get("website") or nl_translation.get("website"),
            "price": price,
            "is_free": event.get("is_free", False)
        }
    
    def get_social_events(self, limit=6):
        """Récupère des événements sociaux spécifiques"""
        events = self.get_events(limit=limit)
        social_events = []
        
        for event in events:
            formatted = self.format_event(event)
            
            # Filtre pour événements sociaux (adaptable)
            if any(keyword in formatted['title'].lower() for keyword in 
                  ['atelier', 'workshop', 'rencontre', 'meeting', 'café', 'débat', 'discussion']):
                social_events.append(formatted)
            else:
                # Inclut tous les événements pour le POC
                social_events.append(formatted)
        
        return social_events

# Test des événements
if __name__ == "__main__":
    print("🎉 API BRUSSELS - VERSION CORRIGÉE!")
    print("=" * 60)
    
    api = BrusselsAPIWorking()
    
    if api.access_token:
        print("✅ Token valide")
        
        # Récupère les événements
        events = api.get_events(page=1, limit=6)
        
        if events:
            print(f"\n📅 AFFICHAGE DE {len(events)} ÉVÉNEMENTS:")
            print("=" * 60)
            
            for i, event in enumerate(events, 1):
                try:
                    formatted_event = api.format_event(event)
                    print(f"\n{i}. {formatted_event['title']}")
                    print(f"   📍 {formatted_event['location']}")
                    print(f"   📅 {formatted_event['start_date']}")
                    if formatted_event['end_date']:
                        print(f"   ➡️  {formatted_event['end_date']}")
                    print(f"   💰 {formatted_event['price']}")
                    if formatted_event['description']:
                        print(f"   📝 {formatted_event['description']}")
                    if formatted_event['url']:
                        print(f"   🔗 {formatted_event['url'][:50]}...")
                    
                except Exception as e:
                    print(f"\n{i}. ❌ Erreur formatage: {e}")
                    # Debug cet événement problématique
                    api.debug_event_structure(event)
                
                if i % 2 == 0:  # Séparateur tous les 2 événements
                    print("-" * 50)
                    
        else:
            print("😞 Aucun événement trouvé")
            
    else:
        print("❌ Token invalide")

🎉 API BRUSSELS - VERSION CORRIGÉE!
✅ Token valide
📊 Statistiques:
   - Page: 1
   - Total pages: 41
   - Total événements: 1009
   - Événements cette page: 25

📅 AFFICHAGE DE 6 ÉVÉNEMENTS:

1. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée
   💰 Payant
   🔗 https://www.art-base.be...

2. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée
   💰 Payant
   📝 La sagesse des petits riens : Un voyage sensible dans l'art de la présence, de la lenteur et des détails, accompagné d'improvisations poétiques au piano.
   🔗 https://www.jardinmusical.org/infos-evenement/la-s...
--------------------------------------------------

3. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée
   💰 Payant
   📝 C'est bientôt, c'est Mardi ! 
Le Effel Quartet, quatuor de saxophones, nous accueille avec un verre offert à partir de 18h30 et nous convie au concert à 19h00.


   🔗 https://chambermusiceurope.org/classique-ici-effel...

4. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée


### Test Premier Agent

In [ ]:

%pip uninstall -y langchain langchain-community langchain-core
%pip install langchain==0.1.16 langchain-community==0.0.38 python-dotenv requests

In [ ]:
%pip install langchain_mistralai

In [6]:
# Version GARANTIE - Custom LLM pour éviter les problèmes d'import
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.llms.base import LLM
from typing import Optional, List, Dict, Any
import requests
import os
import json

class CustomMistralLLM(LLM):
    """Wrapper personnalisé pour l'API Mistral - Évite les problèmes d'import"""
    
    model: str = "mistral-large-latest"
    temperature: float = 0.3
    mistral_api_key: str = ""
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.mistral_api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return f"Erreur API Mistral: {response.status_code}"
        except Exception as e:
            return f"Exception: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "mistral"

print("✅ Custom Mistral LLM importé avec succès!")

✅ Custom Mistral LLM importé avec succès!


In [ ]:
# social_agent_working.py
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.llms.base import LLM
from typing import Optional, List, Dict, Any
import requests
import os
import json
from dotenv import load_dotenv

load_dotenv()

# ========== CUSTOM MISTRAL LLM ==========
class CustomMistralLLM(LLM):
    model: str = "mistral-large-latest"
    temperature: float = 0.3
    mistral_api_key: str = ""
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.mistral_api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return f"Erreur API: {response.status_code}"
        except Exception as e:
            return f"Exception: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "mistral"

# ========== BRUSSELS API ==========
class BrusselsAPI:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, limit=6):
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []
    
    def format_event(self, event):
        translations = event.get("translations", {})
        # Cherche le titre dans n'importe quelle langue
        title = "Activité sans titre"
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("title"):
                title = translations[lang]["title"]
                break
        
        # Même logique pour la description
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("shortdescr"):
                description = translations[lang]["shortdescr"]
                break
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        if dates and isinstance(dates, list) and dates:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
        
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        return {
            "title": title,
            "description": description[:150] + "..." if len(description) > 150 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }

# ========== SOCIAL AGENT ==========
class SocialConnectionAgent:
    def __init__(self):
        # Custom LLM
        self.llm = CustomMistralLLM(mistral_api_key=os.getenv("MISTRAL_API_KEY"))
        
        # API Brussels
        self.api = BrusselsAPI()
        
        # Mémoire
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # Tools
        self.tools = self._setup_tools()
        
        # Agent LangChain
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            agent_kwargs={
                "system_message": SystemMessage(content=self._get_system_prompt()),
                "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")]
            }
        )
    
    def _get_system_prompt(self):
        return """Tu es un assistant bienveillant qui aide les personnes isolées à retrouver du lien social à Bruxelles.
        Sois encourageant et propose des activités concrètes basées sur les événements disponibles."""
    
    def _setup_tools(self):
        def find_social_activities(query: str) -> str:
            events = self.api.get_events(limit=5)
            if not events:
                return "Aucune activité sociale trouvée pour le moment."
            
            result = "🎯 ACTIVITÉS SOCIALES À BRUXELLES:\n\n"
            for i, event in enumerate(events, 1):
                formatted = self.api.format_event(event)
                result += f"{i}. {formatted['title']}\n"
                result += f"   📍 {formatted['location']}\n"
                result += f"   📅 {formatted['start_date']}\n"
                result += f"   💰 {'Gratuit' if formatted['is_free'] else 'Payant'}\n"
                if formatted['description']:
                    result += f"   📝 {formatted['description']}\n"
                result += "\n"
            
            return result
        
        return [
            Tool(
                name="RechercheActivitesSociales",
                func=find_social_activities,
                description="Recherche des activités sociales à Bruxelles pour rencontrer des gens."
            )
        ]
    
    def chat(self, message: str) -> str:
        try:
            response = self.agent.run(message)
            # Gère le cas où la réponse est un dictionnaire
            if isinstance(response, dict) and "response" in response:
                return response["response"]
            elif isinstance(response, dict) and "action_input" in response:
                return str(response)  # Fallback
            return str(response)
        except Exception as e:
            return f"Erreur: {str(e)}"

# ========== TEST ==========
print("🤖 AGENT SOCIAL CONNECTION - FONCTIONNEL!")
print("=" * 60)

agent = SocialConnectionAgent()

# Test simple
test_message = "Bonjour, je me sens seul et j'aimerais rencontrer des gens à Bruxelles"
print(f"👤: {test_message}")
response = agent.chat(test_message)
print(f"🤖: {response}")

### Travailler avec une interface pour tester plus facilement l'agent

In [2]:
import requests
import os
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from dotenv import load_dotenv

load_dotenv()

# ========== CUSTOM MISTRAL LLM ==========
class CustomMistralLLM:
    def __init__(self):
        self.model = "mistral-large-latest"
        self.temperature = 0.3
        self.mistral_api_key = os.getenv("MISTRAL_API_KEY")
    
    def call(self, prompt: str) -> str:
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.mistral_api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return f"Erreur API: {response.status_code}"
        except Exception as e:
            return f"Exception: {str(e)}"

# ========== BRUSSELS API ==========
class BrusselsAPI:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, limit=6):
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []
    
    def format_event(self, event):
        translations = event.get("translations", {})
        
        # Cherche le titre dans n'importe quelle langue
        title = "Activité à Bruxelles"
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("title"):
                title = translations[lang]["title"]
                break
        
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("shortdescr"):
                description = translations[lang]["shortdescr"]
                break
        
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        if dates and isinstance(dates, list) and dates:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
        
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        return {
            "title": title,
            "description": description[:150] + "..." if len(description) > 150 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }

# ========== SOCIAL AGENT SIMPLIFIÉ ==========
class SimpleSocialAgent:
    def __init__(self):
        self.llm = CustomMistralLLM()
        self.api = BrusselsAPI()
        self.conversation_history = []
    
    def chat(self, message: str) -> str:
        # Récupère les événements
        events = self.api.get_events(limit=5)
        events_text = self._format_events(events)
        
        # Crée le contexte complet
        context = f"""
CONTEXTE DES ÉVÉNEMENTS DISPONIBLES:
{events_text}

HISTORIQUE DE CONVERSATION:
{self._format_history()}

MESSAGE ACTUEL: {message}

En tant qu'assistant bienveillant spécialisé dans la connexion sociale, réponds de manière:
- Empathique et encourageante
- En proposant des activités concrètes basées sur les événements disponibles
- En montrant comment chaque activité peut être sociale
- En adaptant aux centres d'intérêt mentionnés
"""
        # Appel à Mistral
        response = self.llm.call(context)
        
        # Met à jour l'historique
        self.conversation_history.append(f"Utilisateur: {message}")
        self.conversation_history.append(f"Assistant: {response}")
        
        # Garde seulement les 6 derniers messages
        if len(self.conversation_history) > 6:
            self.conversation_history = self.conversation_history[-6:]
            
        return response
    
    def _format_events(self, events):
        if not events:
            return "Aucun événement disponible pour le moment."
        
        result = "🎯 ÉVÉNEMENTS À BRUXELLES:\n\n"
        for i, event in enumerate(events, 1):
            formatted = self.api.format_event(event)
            result += f"{i}. {formatted['title']}\n"
            result += f"   📍 {formatted['location']}\n"
            result += f"   📅 {formatted['start_date']}\n"
            result += f"   💰 {'Gratuit' if formatted['is_free'] else 'Payant'}\n"
            if formatted['description']:
                result += f"   📝 {formatted['description']}\n"
            result += "\n"
        
        return result
    
    def _format_history(self):
        if not self.conversation_history:
            return "Aucun historique."
        return "\n".join(self.conversation_history[-4:])  # 2 derniers échanges

print("✅ Agent social chargé avec succès!")

✅ Agent social chargé avec succès!


In [3]:
# Initialisation de l'agent
agent = SimpleSocialAgent()

# Création de l'interface
chat_output = widgets.Output(layout={'border': '1px solid black', 'height': '400px', 'overflow_y': 'scroll', 'padding': '10px'})
message_input = widgets.Text(placeholder='Tapez votre message ici...', layout={'width': '70%'})
send_button = widgets.Button(description='Envoyer ✨', button_style='success', layout={'width': '20%'})
clear_button = widgets.Button(description='Effacer', button_style='warning', layout={'width': '10%'})

# Style CSS
css_style = """
<style>
.chat-message { 
    margin: 10px 0; 
    padding: 12px; 
    border-radius: 15px; 
    max-width: 80%; 
    word-wrap: break-word;
}
.user-message { 
    background: #e3f2fd; 
    margin-left: 20%; 
    border-bottom-right-radius: 5px;
}
.bot-message { 
    background: #f5f5f5; 
    margin-right: 20%; 
    border-bottom-left-radius: 5px;
}
.message-sender { 
    font-weight: bold; 
    margin-bottom: 5px; 
    color: #333;
}
</style>
"""

# Affiche le style CSS
display(HTML(css_style))

# Fonctions
def send_message(_):
    message = message_input.value.strip()
    if not message:
        return
    
    with chat_output:
        # Affiche le message utilisateur
        display(HTML(f"""
        <div class="chat-message user-message">
            <div class="message-sender">👤 Vous</div>
            {message}
        </div>
        """))
    
    # Désactive le bouton pendant le traitement
    send_button.disabled = True
    send_button.description = "Réflexion..."
    
    # Réponse de l'agent
    try:
        response = agent.chat(message)
        with chat_output:
            # Affiche la réponse de l'agent
            display(HTML(f"""
            <div class="chat-message bot-message">
                <div class="message-sender">🤖 Agent Social</div>
                {response.replace(chr(10), '<br>')}
            </div>
            """))
    except Exception as e:
        with chat_output:
            display(HTML(f"""
            <div class="chat-message bot-message">
                <div class="message-sender">❌ Erreur</div>
                Désolé, une erreur s'est produite: {str(e)}
            </div>
            """))
    
    # Réactive le bouton
    send_button.disabled = False
    send_button.description = "Envoyer ✨"
    message_input.value = ''
    
    # Scroll vers le bas
    chat_output.scroll_to_bottom()

def clear_chat(_):
    chat_output.clear_output()
    agent.conversation_history = []
    with chat_output:
        display(HTML("""
        <div class="chat-message bot-message">
            <div class="message-sender">🤖 Agent Social</div>
            Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. 
            Dites-moi ce qui vous intéresse ou comment vous vous sentez aujourd'hui.
        </div>
        """))

def on_enter(_):
    send_message(None)

# Assignation des événements
send_button.on_click(send_message)
clear_button.on_click(clear_chat)
message_input.on_submit(on_enter)

# Affichage de l'interface
display(HTML("<h2>🤖 Social Connection Agent</h2>"))
display(HTML("<p><em>Je vous aide à trouver des activités sociales à Bruxelles 🌟</em></p>"))
display(chat_output)
display(widgets.HBox([message_input, send_button, clear_button]))

# Message de bienvenue initial
with chat_output:
    display(HTML("""
    <div class="chat-message bot-message">
        <div class="message-sender">🤖 Agent Social</div>
        Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. <br><br>
        💡 <strong>Exemples de questions:</strong><br>
        • "Je me sens seul, que faire ?"<br>
        • "Je cherche des activités gratuites"<br>
        • "J'aime la musique, que me conseilles-tu ?"<br>
        • "Quels événements y a-t-il ce weekend ?"
    </div>
    """))

C:\Users\ziani\AppData\Local\Temp\ipykernel_21636\1369923392.py:106: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  message_input.on_submit(on_enter)


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

AttributeError: 'Output' object has no attribute 'scroll_to_bottom'